In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#  Import Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Dense, Dropout, Add, 
                                     MultiHeadAttention, LayerNormalization, GlobalAveragePooling2D)
from tensorflow.keras import Model
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim


# Image Preprocessing

In [2]:
def load_images_from_folder(folder, size=(128, 128)):
    images = []
    for filename in glob.glob(os.path.join(folder, '*.png')):
        img = imread(filename)
        img = img_as_float(img)
        img_resized = tf.image.resize(img, size).numpy()
        images.append(img_resized)
    return np.array(images)

def load_and_preprocess_datasets(base_folder):
    rainy_images = []
    clear_images = []
    for i in range(6):  # Assuming 6 datasets
        dataset_folder = os.path.join(base_folder, f'dataset_{i}')
        rainy_folder = os.path.join(dataset_folder, 'rainy')
        clear_folder = os.path.join(dataset_folder, 'clear')
        
        rainy_images.extend(load_images_from_folder(rainy_folder))
        clear_images.extend(load_images_from_folder(clear_folder))
        
    return np.array(rainy_images), np.array(clear_images)

base_folder = 'path_to_your_dataset_folder'
rainy_images, clear_images = load_and_preprocess_datasets(base_folder)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/rainydata/RainStreak/rain_streak'

# Advanced Dynamic Pyramid Model

In [4]:
# Advanced Dynamic Pyramid Model with Multi-Scale Feature Extraction
def advanced_dynamic_pyramid_model(input_shape, num_scales=3):
    inputs = Input(shape=input_shape)
    pyramid_features = []
    
    for scale in range(num_scales):
        scale_factor = 2 ** scale
        x_scaled = tf.image.resize(inputs, [input_shape[0] // scale_factor, input_shape[1] // scale_factor])
        
        # Dynamic kernel size based on scale
        kernel_size = 3 + scale
        x_conv = Conv2D(64 * scale_factor, (kernel_size, kernel_size), activation='relu', padding='same')(x_scaled)
        x_conv = Add()([x_conv, x_scaled])  # Residual connection
        pyramid_features.append(x_conv)

    fused_features = tf.concat(pyramid_features, axis=-1)  # Concatenate along channel dimension
    return inputs, fused_features


# Window Partition and Reverse Functions

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Add, LayerNormalization, MultiHeadAttention, Dense
from tensorflow.image import extract_patches

# Function to partition windows
def window_partition(x, window_size):
    """Partition feature map into non-overlapping windows of size window_size."""
    patches = extract_patches(images=x,
                              sizes=[1, window_size, window_size, 1],
                              strides=[1, window_size, window_size, 1],
                              rates=[1, 1, 1, 1],
                              padding='VALID')
    return patches

# Function to reverse windows back into the original shape
def window_reverse(patches, window_size, input_shape):
    """Reconstruct the feature map from partitioned windows."""
    num_windows = (input_shape[1] // window_size) * (input_shape[2] // window_size)
    reshaped_patches = tf.reshape(patches, [num_windows, window_size, window_size, -1])
    reconstructed = tf.reshape(reshaped_patches, input_shape)
    return reconstructed


# Patch Merging Function

In [ ]:
# Patch Merging Function
def patch_merge(x, window_size):
    """Merge adjacent patches to create a larger feature window."""
    # Extract patches (patch size = window size)
    patches = window_partition(x, window_size)
    
    # Calculate the new merged patch (by averaging or pooling)
    merged_patches = tf.reduce_mean(patches, axis=-1, keepdims=True)
    
    # Reshape into the original window size
    new_window_size = window_size * 2
    merged = window_reverse(merged_patches, new_window_size, x.shape)
    
    return merged, new_window_size


# Modified Swin Transformer Block with Recursion and Iteration

In [ ]:
# Modified Swin Transformer Block with Recursive Patch Merging and Feature Extraction
def modified_swin_transformer_block(fused_features, initial_window_size=4, num_heads=4, key_dim=64, num_recursions=6):
    def recursive_block(x, window_size, iteration, recursion):
        if recursion == 0:
            return x
        
        for _ in range(iteration):
            # Partition the feature map into windows
            windows = window_partition(x, window_size)
            window_shape = (windows.shape[-2], windows.shape[-1])

            # Reshape windows to apply multi-head attention
            windows_reshaped = tf.reshape(windows, (-1, window_shape[0] * window_shape[1], fused_features.shape[-1]))

            # Layer Normalization before Attention
            x_norm = LayerNormalization()(windows_reshaped)

            # Multi-Head Self Attention within windows
            attention = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x_norm, x_norm)

            # Apply thresholding to attention scores
            attention_scores = tf.reduce_mean(tf.abs(attention), axis=-1, keepdims=True)
            threshold = 0.1  # Example threshold for pruning attention scores
            attention = tf.where(attention_scores > threshold, attention, tf.zeros_like(attention))

            # Residual connection
            x_add = Add()([windows_reshaped, attention])

            # Reverse the window partitioning
            x_reconstructed = window_reverse(x_add, window_size, x.shape)

            # Layer Normalization before Feed-Forward Network
            x_norm_ffn = LayerNormalization()(x_reconstructed)
            x_ffn = Dense(128, activation='relu')(x_norm_ffn)
            x_ffn_out = Dense(fused_features.shape[-1])(x_ffn)

            # Residual connection
            x_out = Add()([x_reconstructed, x_ffn_out])

            # Patch merging to increase window size
            x_out, window_size = patch_merge(x_out, window_size)

        return recursive_block(x_out, window_size, iteration, recursion - 1)
    
    # Start recursive process
    x_out = recursive_block(fused_features, initial_window_size, 2, num_recursions)
    return x_out


# Image Restoration and Enhancement

In [6]:
# Image Restoration and Enhancement Block for final image reconstruction
def image_restoration_and_enhancement(x_out, num_classes=3):
    """
    Image Restoration and Enhancement Block for image reconstruction.
    - num_classes: Number of output channels (3 for RGB)
    """
    # Layer normalization before enhancement process
    x = LayerNormalization()(x_out)

    # Enhancement with convolutional layers
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    
    # Upsampling for restoring spatial resolution
    x = UpSampling2D(size=(2, 2))(x)

    # Additional convolution for finer restoration
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    
    # Layer normalization before final processing
    x = LayerNormalization()(x)
    
    # Final output layer to reconstruct the image with RGB channels
    outputs = Conv2D(num_classes, (3, 3), padding='same', activation='tanh')(x)  # 'tanh' for pixel values in [-1, 1]

    return outputs


# Construct and Compile the Model

In [ ]:
# Import required layers and model creation utilities
from tensorflow.keras.models import Model

# Full Model Construction
def build_full_model(input_shape):
    """
    Construct the full model by integrating dynamic pyramid, recursive modified Swin Transformer,
    and image restoration blocks.
    
    Parameters:
    - input_shape: Shape of the input image (height, width, channels)
    
    Returns:
    - Model: Full TensorFlow/Keras model ready for training and testing
    """
    # Step 1: Input layer
    inputs = Input(shape=input_shape)
    
    # Step 2: Advanced Dynamic Pyramid for Multi-Scale Feature Extraction
    inputs, pyramid_features = advanced_dynamic_pyramid_model(input_shape=input_shape, num_scales=3)
    
    # Step 3: Modified Swin Transformer with Recursive Patch Merging (6 recursions, 2 iterations per recursion)
    transformer_output = modified_swin_transformer_block(fused_features=pyramid_features, 
                                                         initial_window_size=4, num_heads=4, key_dim=64, num_recursions=6)
    
    # Step 4: Image Restoration and Enhancement
    outputs = image_restoration_and_enhancement(x_out=transformer_output, num_classes=3)
    
    # Step 5: Model creation
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

# Compile the model with appropriate optimizer and loss function
input_shape = (256, 256, 3)  # Example input shape (height, width, channels)
model = build_full_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Summary of the model
model.summary()


# Model Training

In [ ]:
# Import necessary libraries for training
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

# Set up input shape and define the model
input_shape = (256, 256, 3)
model = build_full_model(input_shape)

# Compile the model with Adam optimizer and MSE loss
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Define callbacks for saving the best model and early stopping
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define training parameters
epochs = 50
batch_size = 16

# Train the model
history = model.fit(train_data, train_labels,
                    validation_data=(val_data, val_labels),
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[checkpoint, early_stopping])

# Save the final model
model.save('final_model.h5')


# Model Evaluation

In [ ]:
# Import necessary libraries for evaluation
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('best_model.h5')

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)

# Print the test results
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


# PSNR and SSIM Calculation

In [ ]:
# Import necessary libraries for PSNR and SSIM calculations
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import numpy as np

# Function to calculate PSNR
def calculate_psnr(ground_truth, prediction):
    return peak_signal_noise_ratio(ground_truth, prediction, data_range=1.0)

# Function to calculate SSIM
def calculate_ssim(ground_truth, prediction):
    return structural_similarity(ground_truth, prediction, multichannel=True, data_range=1.0)

# Use the trained model to predict on the test data
predictions = model.predict(test_data)

# Initialize lists to store PSNR and SSIM values
psnr_list = []
ssim_list = []

# Loop through each test image and calculate PSNR and SSIM
for i in range(len(test_data)):
    gt_image = test_labels[i]
    pred_image = predictions[i]
    
    psnr = calculate_psnr(gt_image, pred_image)
    ssim = calculate_ssim(gt_image, pred_image)
    
    psnr_list.append(psnr)
    ssim_list.append(ssim)

# Calculate average PSNR and SSIM
average_psnr = np.mean(psnr_list)
average_ssim = np.mean(ssim_list)

# Print the results
print(f"Average PSNR: {average_psnr:.4f}")
print(f"Average SSIM: {average_ssim:.4f}")


# Prediction and Evaluation on Test Images

In [ ]:
# Import necessary libraries for visualization
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('best_model.h5')

# Predict on test data
predictions = model.predict(test_data)

# Visualize the results
num_images_to_display = 3

for i in range(num_images_to_display):
    gt_image = test_labels[i]
    pred_image = predictions[i]
    
    # Display ground truth and prediction side by side
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.title("Ground Truth")
    plt.imshow((gt_image * 255).astype(np.uint8))  # Convert to range [0, 255] for display
    
    plt.subplot(1, 2, 2)
    plt.title("Predicted")
    plt.imshow((pred_image * 255).astype(np.uint8))  # Convert to range [0, 255] for display
    
    plt.show()


# PSNR and SSIM Evaluation Across Test Set

In [ ]:
# PSNR and SSIM Evaluation Across the Test Set
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# Function to evaluate PSNR and SSIM for the entire test set
def evaluate_test_set_psnr_ssim(test_data, test_labels, predictions):
    psnr_values = []
    ssim_values = []

    for i in range(len(test_data)):
        gt_image = test_labels[i]
        pred_image = predictions[i]
        
        psnr = peak_signal_noise_ratio(gt_image, pred_image, data_range=1.0)
        ssim = structural_similarity(gt_image, pred_image, multichannel=True, data_range=1.0)
        
        psnr_values.append(psnr)
        ssim_values.append(ssim)

    # Compute the average PSNR and SSIM
    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)

    print(f"Average PSNR: {avg_psnr:.4f}")
    print(f"Average SSIM: {avg_ssim:.4f}")
    
    return avg_psnr, avg_ssim

# Get predictions from the model
predictions = model.predict(test_data)

# Evaluate PSNR and SSIM on the test set
avg_psnr, avg_ssim = evaluate_test_set_psnr_ssim(test_data, test_labels, predictions)


# Visualization of Training History

In [ ]:
# Import libraries for visualization
import matplotlib.pyplot as plt

# Plot the training and validation loss
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Show the plots
plt.show()
